 В качестве набора данных я выбрала тот же самый датасет, что и был выбран мной в предыдущих лабораторных работах. Следовательно вся предобработка данных будет схожа с предобработкой в предыдущих лабах. 

In [0]:
#Импорт библиотек 
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

In [2]:
#Монтирую гугл диск, чтобы взять оттуда датасет
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Загружаю данные с гугл диска
data = pd.read_csv('/content/gdrive/My Drive/winequality-red.csv', sep=",")

In [4]:
data.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
cols_x = ['fixed acidity', 'citric acid', 'sulphates', 'alcohol']   
# целевой признак
col_y = 'quality'
#разделение данных  на тестовую и обучающую выборки 
X_train, X_test, y_train, y_test = train_test_split(data[cols_x], data[col_y], test_size = 0.5, random_state = 11)
N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

799 800


# Модель бэггинга

In [0]:
bag = BaggingClassifier(DecisionTreeClassifier(random_state=1), n_estimators=10).fit(X_train, y_train)

In [0]:
target_bag = bag.predict(X_test)

In [0]:
def convert_target_to_binary(array:np.ndarray, target:int) -> np.ndarray:
    # Если целевой признак совпадает с указанным, то 1 иначе 0
    res = [1 if x==target else 0 for x in array]
    return res

In [9]:
# Если целевой признак == 6, 
# то будем считать этот случай 1 в бинарном признаке
bin_y_test = convert_target_to_binary(y_test, 6)
list(zip(y_test, bin_y_test))[0:15]

[(5, 0),
 (5, 0),
 (5, 0),
 (6, 1),
 (6, 1),
 (5, 0),
 (7, 0),
 (5, 0),
 (5, 0),
 (6, 1),
 (5, 0),
 (6, 1),
 (5, 0),
 (3, 0),
 (6, 1)]

In [0]:
# Конвертация предсказанных признаков
bin_target_bag = convert_target_to_binary(target_bag, 6)

In [11]:
ac_bag = accuracy_score(y_test, target_bag)
bas_bag = balanced_accuracy_score(bin_y_test, bin_target_bag)
rec_bag = recall_score(bin_y_test, bin_target_bag)
print('''accuracy_score: {0}
balanced_accuracy_score: {1}
recall_score: {2}'''.format(ac_bag, bas_bag, rec_bag))

accuracy_score: 0.6
balanced_accuracy_score: 0.6507403380704615
recall_score: 0.5398773006134969


#Случайный лес

Рекомендуется в задачах классификации брать dl=D‾‾√
, а в задачах регрессии dl=D/3
. В библиотеке scikit-learn это признак max_features

In [12]:
forest = RandomForestClassifier(random_state=1, max_features = 2).fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
target_forest = forest.predict(X_test)

In [0]:
# Конвертация предсказанных признаков
bin_target_forest = convert_target_to_binary(target_forest, 6)

In [15]:
ac_forest = accuracy_score(y_test, target_forest)
bas_forest = balanced_accuracy_score(bin_y_test, bin_target_forest)
rec_forest = recall_score(bin_y_test, bin_target_forest)
print('''accuracy_score: {0}
balanced_accuracy_score: {1}
recall_score: {2}'''.format(ac_forest, bas_forest, rec_forest))

accuracy_score: 0.5825
balanced_accuracy_score: 0.6229258885351143
recall_score: 0.5644171779141104


#Подбор гиперпараметра с использованием GridSearchCV и кросс-валидации 

## Для бэггинга

In [30]:
n_range = np.array(range(1,202,10))
tuned_parameters = [{'n_estimators': n_range}]
tuned_parameters

[{'n_estimators': array([  1,  11,  21,  31,  41,  51,  61,  71,  81,  91, 101, 111, 121,
         131, 141, 151, 161, 171, 181, 191, 201])}]

In [31]:
bag_gs = GridSearchCV(BaggingClassifier(DecisionTreeClassifier()), tuned_parameters, cv=KFold(n_splits=3), scoring='accuracy')
bag_gs.fit(data[cols_x], data[col_y])

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            ...stimators=10, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': array([  1,  11,  21,  31,  41,  51,  61,  71,  81,  91, 101, 111, 121,
       131, 141, 151, 161, 171, 181, 191, 201])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [32]:
bag_gs.best_params_

{'n_estimators': 51}

In [0]:
bag_gs.best_estimator_.fit(X_train, y_train)
target_bag_gs = bag_gs.best_estimator_.predict(X_test)

In [0]:
# Конвертация предсказанных признаков
bin_target_bag_gs = convert_target_to_binary(target_bag_gs, 6)

In [36]:
ac_bag_gs = accuracy_score(y_test, target_bag_gs)
bas_bag_gs = balanced_accuracy_score(bin_y_test, bin_target_bag_gs)
rec_bag_gs = recall_score(bin_y_test, bin_target_bag_gs)
print('''accuracy_score: {0}
balanced_accuracy_score: {1}
recall_score: {2}'''.format(ac_bag_gs, bas_bag_gs, rec_bag_gs))

accuracy_score: 0.60875
balanced_accuracy_score: 0.6555292381765939
recall_score: 0.5705521472392638


## Для случайного леса

In [38]:
n_range2 = np.array(range(1,5,1))
tuned_parameters2 = [{'max_features': n_range2}]
tuned_parameters2

[{'max_features': array([1, 2, 3, 4])}]

In [0]:
forest_gs = GridSearchCV(RandomForestClassifier(), tuned_parameters2, cv=KFold(n_splits=10), scoring='accuracy')
forest_gs.fit(data[cols_x], data[col_y])

In [41]:
forest_gs.best_params_

{'max_features': 1}

In [0]:
forest_gs.best_estimator_.fit(X_train, y_train)
target_forest_gs = bag_gs.best_estimator_.predict(X_test)

In [0]:
# Конвертация предсказанных признаков
bin_target_forest_gs = convert_target_to_binary(target_forest_gs, 6)

In [44]:
ac_forest_gs = accuracy_score(y_test, target_forest_gs)
bas_forest_gs = balanced_accuracy_score(bin_y_test, bin_target_forest_gs)
rec_forest_gs = recall_score(bin_y_test, bin_target_forest_gs)
print('''accuracy_score: {0}
balanced_accuracy_score: {1}
recall_score: {2}'''.format(ac_forest_gs, bas_forest_gs, rec_forest_gs))

accuracy_score: 0.60875
balanced_accuracy_score: 0.6555292381765939
recall_score: 0.5705521472392638
